In [1]:
import numpy as np
import warnings
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
from torch.distributions import constraints
from pyro.optim import ClippedAdam
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

import time
import matplotlib.pyplot as plt
from itertools import groupby

warnings.filterwarnings("ignore")

In [2]:
# Copyright (c) 2017-2019 Uber Technologies, Inc.
# SPDX-License-Identifier: Apache-2.0

import torch
from torch.distributions import constraints

from pyro.contrib.gp.kernels.kernel import Kernel
from pyro.nn.module import PyroParam


def _torch_sqrt(x, eps=1e-12):
    """
    A convenient function to avoid the NaN gradient issue of :func:`torch.sqrt`
    at 0.
    """
    # Ref: https://github.com/pytorch/pytorch/issues/2421
    return (x + eps).sqrt()


class Isotropy(Kernel):
    """
    Base class for a family of isotropic covariance kernels which are functions of the
    distance :math:`|x-z|*l`, where :math:`l` is the length-scale parameter.

    By default, the parameter ``lengthscale`` has size 1. To use the isotropic version
    (different lengthscale for each dimension), make sure that ``lengthscale`` has size
    equal to ``input_dim``.

    :param torch.Tensor lengthscale: Length-scale parameter of this kernel.
    """

    def __init__(self, input_dim, variance=None, lengthscale=None, active_dims=None):
        super().__init__(input_dim, active_dims)

        variance = torch.tensor(1.0) if variance is None else variance
        self.variance = PyroParam(variance, constraints.positive)

        lengthscale = torch.tensor(1.0) if lengthscale is None else lengthscale
        self.lengthscale = PyroParam(lengthscale, constraints.positive)

    def _square_scaled_dist(self, X, Z=None):
        r"""
        Returns :math:`\|\frac{X-Z}{l}\|^2`.
        """
        if Z is None:
            Z = X
        X = self._slice_input(X)
        Z = self._slice_input(Z)
        if X.size(1) != Z.size(1):
            raise ValueError("Inputs must have the same number of features.")

        scaled_X = X * self.lengthscale
        scaled_Z = Z * self.lengthscale
        X2 = (scaled_X ** 2).sum(1, keepdim=True)
        Z2 = (scaled_Z ** 2).sum(1, keepdim=True)
        XZ = scaled_X.matmul(scaled_Z.t())
        r2 = X2 - 2 * XZ + Z2.t()
        return r2.clamp(min=0)

    def _scaled_dist(self, X, Z=None):
        r"""
        Returns :math:`l * \|X-Z\|`.
        """
        return _torch_sqrt(self._square_scaled_dist(X, Z))

    def _diag(self, X):
        """
        Calculates the diagonal part of covariance matrix on active features.
        """
        return self.variance.expand(X.size(0))


class INV_RBF(Isotropy):
    r"""
    Implementation of Radial Basis Function kernel:

        :math:`k(x,z) = \sigma^2\exp\left(-0.5 \times \frac{|x-z|^2}{l^2}\right).`

    .. note:: This kernel also has name `Squared Exponential` in literature.
    """

    def __init__(self, input_dim, variance=None, lengthscale=None, active_dims=None):
        super().__init__(input_dim, variance, lengthscale, active_dims)

    def forward(self, X, Z=None, diag=False):
        if diag:
            return self._diag(X)

        r2 = self._square_scaled_dist(X, Z)
        return self.variance * torch.exp(-0.5 * r2)

In [3]:
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, brier_score_loss

def Predict_GPC(GPC, X_test,y_test):
    with torch.no_grad():
        f_loc, f_scale = GPC(X_test)
        p_test = GPC.likelihood.response_function(f_loc).cpu().numpy().astype("float64")
    print(" Test sample size: ", len(p_test))
    nll = log_loss(y_test.cpu(), p_test) #negative log likelihood
    auc = roc_auc_score(y_test.cpu(), p_test) #AUC
    brier_score = brier_score_loss(y_test.cpu(), p_test) #brier_score_loss
    y_pred = np.zeros(len(p_test))
    y_pred[p_test>= 0.5] = 1
    accuracy = np.sum(y_test.cpu().numpy() == y_pred)/len(y_test.cpu().numpy())
    print(" accuracy: {} \n negative log likelihood: {} \n brier_score_loss:{} \n AUC: {}".format(accuracy, nll, brier_score, auc))
    return np.array([accuracy, nll, brier_score, auc]),p_test

In [4]:
X=np.load('EEG_X.npy')

In [5]:
y=np.load('EEG_y.npy')
y=np.array(y)

In [6]:
n = len(X[0][0])
n

122

In [7]:
X.shape

(256, 57, 122)

In [8]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

In [9]:
%%time
L_cv = []
res_cv = []
y_cv = []
cv = 0
for ind_train, ind_test in kf.split(X=np.zeros([122,57]), y=y):
    print('Cross Validation:',cv)
    cv+=1
    L = []
    L_exp = []
    y_predict = []
    np.random.seed(0)
    pyro.set_rng_seed(0)
    t = 0
    for data in X:
        L_0 = []
        if t%30==0:
            print("{}th time".format(t))
        t+=1
        data = np.transpose(data)
        X_train, y_train = data[ind_train], y[ind_train]
        X_test, y_test = data[ind_test], y[ind_test]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        pca = PCA()
        pca_x = pca.fit_transform(X_train)[:,-20:]
        pca_x = scaler.fit_transform(pca_x)

    #     X_train = torch.tensor(X_train,dtype= torch.float32)
    #     y_train = torch.tensor(y_train, dtype=torch.float32).squeeze()
    #     X_test = torch.tensor(X_test,dtype= torch.float32)
    #     y_test = torch.tensor(y_test, dtype=torch.float32).squeeze()

        # Choose kernel and likelihood for GP
        for random_seed in range(20):
    #         np.random.seed(random_seed)
            pyro.set_rng_seed(random_seed)
            N = X_train.shape[0]
            X_ = np.insert(X_train,0,pca_x[:,random_seed],axis=1)
            p = X_.shape[1]
            X_ = torch.tensor(X_)
            y_samples = torch.tensor(y_train, dtype=torch.float32).squeeze()
            kernel = INV_RBF(input_dim = p, variance = torch.tensor(1.),lengthscale = torch.ones(p))
            likelihood = gp.likelihoods.Binary()
            gpc = gp.models.VariationalGP(X_,y_samples,kernel=kernel,jitter = torch.tensor(1e-06), 
                                        likelihood=likelihood,
                                        whiten =True)
            gpc.kernel.lengthscale =  pyro.nn.PyroSample(dist.Exponential(torch.tensor([2.0])).expand([p]).to_event(1))
            gpc.kernel.variance = pyro.nn.PyroSample(dist.LogNormal(torch.tensor(0.0), torch.tensor(2.0)))
            num_steps = 1000
            initial_lr = 0.005
            gamma = 0.5  # final learning rate will be gamma * initial_lr
            lrd = gamma ** (1 / num_steps)
            optim = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})

            svi = SVI(gpc.model,gpc.guide,optim,loss=Trace_ELBO())
            losses =np.zeros(num_steps)

            pyro.clear_param_store()
            for iteration in range(num_steps):
                losses[iteration]=svi.step()

            L_0.append(pyro.param('kernel.lengthscale_map').data.cpu().numpy())

        L.append(L_0)
        L_0 = np.array(L_0)
        threshold = np.quantile(L_0[:,0],0.6)

        selected_var = np.quantile(L_0[:,1:],0.5,axis = 0)>=threshold
        L_exp.append(selected_var)

        if sum(selected_var)==0:
            max_ind = np.argmax(np.quantile(L_0[:,1:],0.5,axis = 0))
            selected_var[max_ind]=True

        X_selected = X_train[:,selected_var]
        X_test = X_test[:,selected_var]
        kernel = 1.0 * RBF(np.ones(X_selected.shape[1]))
        gpc = GaussianProcessClassifier(kernel=kernel,
                 random_state=0).fit(X_selected, y_train)
        y_predict.append(gpc.predict(X_test))
    
    L_cv.append(L)
    y_predict = np.array(y_predict)
    res = []
    for ind in range(y_predict.shape[1]):
        a = y_predict[:,ind]
        lst = []
        for n,c in groupby(a):

            num,count = n,sum(1 for i in c)
            lst.append([num,count])

        lst = np.array(lst)
        res.append(lst[np.argmax(lst[:,1])][0])
    res_cv.append(res)
    y_cv.append(y_test)


Cross Validation: 0
0th time
30th time
60th time
90th time
120th time
150th time
180th time
210th time
240th time
Cross Validation: 1
0th time
30th time
60th time
90th time
120th time
150th time
180th time
210th time
240th time
Cross Validation: 2
0th time
30th time
60th time
90th time
120th time
150th time
180th time
210th time
240th time
Cross Validation: 3
0th time
30th time
60th time
90th time
120th time
150th time
180th time
210th time
240th time
Cross Validation: 4
0th time
30th time
60th time
90th time
120th time
150th time
180th time
210th time
240th time
CPU times: user 1d 4h 7min 10s, sys: 23min 15s, total: 1d 4h 30min 26s
Wall time: 1d 3h 54min 20s


In [10]:
y_cv = np.array(y_cv)
L_cv = np.array(L_cv)
res_cv = np.array(res_cv)

In [11]:
np.save("L_cv_pca_0307.npy", L_cv)

In [12]:
np.save("y_cv_pca_0307.npy", y_cv)

In [13]:
np.save("res_cv_pca_0307.npy", res_cv)

In [14]:
%%time
cv = 0
res_cv = []
for ind_train, ind_test in kf.split(X=np.zeros([122,57]), y=y):
    print('Cross Validation:',cv)
    
    y_predict = []
    np.random.seed(0)
    pyro.set_rng_seed(0)
    t = 0
    for data in X:
        L_0 = []
        if t%50==0:
            print("{}th time".format(t))
        
        data = np.transpose(data)
        X_train, y_train = data[ind_train], y[ind_train]
        X_test, y_test = data[ind_test], y[ind_test]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        L_0 = np.array(L_cv[cv][t])
        threshold = np.quantile(L_0[:,0],0.6)

        selected_var = np.quantile(L_0[:,1:],0.5,axis = 0)>=threshold

        if sum(selected_var)==0:
            max_ind = np.argmax(np.quantile(L_0[:,1:],0.5,axis = 0))
            selected_var[max_ind]=True

        X_selected = X_train[:,selected_var]
        X_test = X_test[:,selected_var]
        kernel = 1.0 * RBF(np.ones(X_selected.shape[1]))
        gpc = GaussianProcessClassifier(kernel=kernel,
                 random_state=0).fit(X_selected, y_train)
        y_predict.append(gpc.predict(X_test))
        
        t+=1
        
    y_predict = np.array(y_predict)
    res = []
    for ind in range(y_predict.shape[1]):
        a = y_predict[:,ind]
        lst = []
        for n,c in groupby(a):

            num,count = n,sum(1 for i in c)
            lst.append([num,count])

        lst = np.array(lst)
        res.append(lst[np.argmax(lst[:,1])][0])
    res_cv.append(res)
    cv+=1


Cross Validation: 0
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 1
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 2
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 3
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 4
0th time
50th time
100th time
150th time
200th time
250th time
CPU times: user 5min 53s, sys: 1min 13s, total: 7min 6s
Wall time: 54.7 s


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
selected_acc = []
selected_f1 = []
selected_tpr = []
selected_fpr = []

for j in range(len(res_cv)):
    selected_acc.append(accuracy_score(y_cv[j], res_cv[j]))
    selected_f1.append(f1_score(y_cv[j], res_cv[j], average='weighted'))
    tpr = sum((np.array(y_cv[j]).squeeze()+np.array(res_cv[j])) == 2)/sum(np.array(y_cv[j]).squeeze())
    selected_tpr.append(tpr)
    fpr = sum((np.array(y_cv[j]).squeeze()-np.array(res_cv[j])) == -1)/(len(np.array(y_cv[j]).squeeze()) - sum(np.array(y_cv[j]).squeeze()))
    selected_fpr.append(fpr)

In [17]:
np.mean(selected_acc), np.std(selected_acc)

(0.704, 0.15297276446043154)

In [21]:
%%time
cv = 0
res_cv = []
for ind_train, ind_test in kf.split(X=np.zeros([122,57]), y=y):
    print('Cross Validation:',cv)
    
    y_predict = []
    np.random.seed(0)
    pyro.set_rng_seed(0)
    t = 0
    for data in X:
        L_0 = []
        if t%50==0:
            print("{}th time".format(t))
        
        data = np.transpose(data)
        X_train, y_train = data[ind_train], y[ind_train]
        X_test, y_test = data[ind_test], y[ind_test]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        L_0 = np.array(L_cv[cv][t])
        threshold = np.quantile(L_0[:,0],0.55)

        selected_var = np.quantile(L_0[:,1:],0.5,axis = 0)>=threshold

        if sum(selected_var)==0:
            max_ind = np.argmax(np.quantile(L_0[:,1:],0.5,axis = 0))
            selected_var[max_ind]=True

        X_selected = X_train[:,selected_var]
        X_test = X_test[:,selected_var]
        kernel = 1.0 * RBF(np.ones(X_selected.shape[1]))
        gpc = GaussianProcessClassifier(kernel=kernel,
                 random_state=0).fit(X_selected, y_train)
        y_predict.append(gpc.predict(X_test))
        
        t+=1
        
    y_predict = np.array(y_predict)
    res = []
    for ind in range(y_predict.shape[1]):
        a = y_predict[:,ind]
        lst = []
        for n,c in groupby(a):

            num,count = n,sum(1 for i in c)
            lst.append([num,count])

        lst = np.array(lst)
        res.append(lst[np.argmax(lst[:,1])][0])
    res_cv.append(res)
    cv+=1


Cross Validation: 0
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 1
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 2
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 3
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 4
0th time
50th time
100th time
150th time
200th time
250th time
CPU times: user 8min 58s, sys: 1min 52s, total: 10min 50s
Wall time: 1min 23s


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
selected_acc = []
selected_f1 = []
selected_tpr = []
selected_fpr = []

for j in range(len(res_cv)):
    selected_acc.append(accuracy_score(y_cv[j], res_cv[j]))
    selected_f1.append(f1_score(y_cv[j], res_cv[j], average='weighted'))
    tpr = sum((np.array(y_cv[j]).squeeze()+np.array(res_cv[j])) == 2)/sum(np.array(y_cv[j]).squeeze())
    selected_tpr.append(tpr)
    fpr = sum((np.array(y_cv[j]).squeeze()-np.array(res_cv[j])) == -1)/(len(np.array(y_cv[j]).squeeze()) - sum(np.array(y_cv[j]).squeeze()))
    selected_fpr.append(fpr)

In [24]:
np.mean(selected_acc), np.std(selected_acc)

(0.7366666666666667, 0.16639644761165362)

In [28]:
%%time
cv = 0
res_cv = []
for ind_train, ind_test in kf.split(X=np.zeros([122,57]), y=y):
    print('Cross Validation:',cv)
    
    y_predict = []
    np.random.seed(0)
    pyro.set_rng_seed(0)
    t = 0
    for data in X:
        L_0 = []
        if t%50==0:
            print("{}th time".format(t))
        
        data = np.transpose(data)
        X_train, y_train = data[ind_train], y[ind_train]
        X_test, y_test = data[ind_test], y[ind_test]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        L_0 = np.array(L_cv[cv][t])
        threshold = np.quantile(L_0[:,0],0.5)

        selected_var = np.quantile(L_0[:,1:],0.5,axis = 0)>=threshold

        if sum(selected_var)==0:
            max_ind = np.argmax(np.quantile(L_0[:,1:],0.5,axis = 0))
            selected_var[max_ind]=True

        X_selected = X_train[:,selected_var]
        X_test = X_test[:,selected_var]
        kernel = 1.0 * RBF(np.ones(X_selected.shape[1]))
        gpc = GaussianProcessClassifier(kernel=kernel,
                 random_state=0).fit(X_selected, y_train)
        y_predict.append(gpc.predict(X_test))
        
        t+=1
        
    y_predict = np.array(y_predict)
    res = []
    for ind in range(y_predict.shape[1]):
        a = y_predict[:,ind]
        lst = []
        for n,c in groupby(a):

            num,count = n,sum(1 for i in c)
            lst.append([num,count])

        lst = np.array(lst)
        res.append(lst[np.argmax(lst[:,1])][0])
    res_cv.append(res)
    cv+=1


Cross Validation: 0
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 1
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 2
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 3
0th time
50th time
100th time
150th time
200th time
250th time
Cross Validation: 4
0th time
50th time
100th time
150th time
200th time
250th time
CPU times: user 13min 55s, sys: 2min 48s, total: 16min 43s
Wall time: 2min 7s


In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
selected_acc = []
selected_f1 = []
selected_tpr = []
selected_fpr = []

for j in range(len(res_cv)):
    selected_acc.append(accuracy_score(y_cv[j], res_cv[j]))
    selected_f1.append(f1_score(y_cv[j], res_cv[j], average='weighted'))
    tpr = sum((np.array(y_cv[j]).squeeze()+np.array(res_cv[j])) == 2)/sum(np.array(y_cv[j]).squeeze())
    selected_tpr.append(tpr)
    fpr = sum((np.array(y_cv[j]).squeeze()-np.array(res_cv[j])) == -1)/(len(np.array(y_cv[j]).squeeze()) - sum(np.array(y_cv[j]).squeeze()))
    selected_fpr.append(fpr)

In [31]:
np.mean(selected_acc), np.std(selected_acc)

(0.7453333333333333, 0.14327673301070984)